_Run the first 2 code cells without modifications_

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import Markdown, Image
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd
from pathlib import Path
from pprint import pprint as ptp
import sys
import time

#pd.set_option("display.max_colwidth", 200)
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-v0_8-muted')

print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
print('Currrent dir: {}\n'.format(Path.cwd()))


def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """

    if up:
        newp = str(Path(this_path).parent)
    else:
        newp = str(Path(this_path))
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))


def fdir(obj, start_with_str='_', exclude=True):
    """Filtered dir() for method discovery."""
    return [d for d in dir(obj) if not d.startswith(start_with_str) == exclude]

def despine(which=['top','right']):
    """which ([str])): 'left','top','right','bottom'."""

    ax = plt.gca()
    for side in which:
        ax.spines[side].set_visible(False)
    return

def md_width_comment(w:int=120) -> str:
    """Width guide for composing md documents."""
    return f"<!-- dotted line width = {w}\n{'.'*w}-->"


# autoreload extension
%load_ext autoreload
%autoreload 2

Python ver: 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:38:13) [GCC 12.3.0]
Python env: rpt
Currrent dir: /home/cat/projects/MCCE_ProtInfo/notebooks



---
---

In [2]:
from argparse import Namespace
import Bio.PDB
from collections import defaultdict
from protinfo import USER_MCCE
from protinfo import info, log_parser, run
from protinfo.io_utils import subprocess_run, subprocess
import requests
import shutil
from typing import Union
import warnings

In [3]:
USER_MCCE

'/home/cat/miniconda3/envs/rpt/bin/mcce'

In [105]:
cif_file = Path("6kig-assembly1.cif")
protname = iou.get_cif_protname(cif_file)
#pdb_out = iou.cif2pdb(cif_file)


'HEADER    6kig-assembly1.cif converted to pdb by MCCE_ProtInfo; truncated: True\nTITLE     "Structure of cyanobacterial photosystem I-IsiA supercomplex" \n'

HEADER    6kig-assembly1.cif converted to pdb by MCCE_ProtInfo; truncated: True
TITLE     "Structure of cyanobacterial photosystem I-IsiA supercomplex" 


In [127]:
pdb_from_cif_fp = Path("6kig.pdb")
#insert_pdb_hdr(pdb_from_cif_fp, hdr)
pdb_hdr_d = Bio.PDB.parse_pdb_header(pdb_from_cif_fp)

if "truncated" in pdb_hdr_d.get("head"):
    pdb_prot = f"{pdb_hdr_d.get('name').title()}\nNOTE: {pdb_hdr_d.get('head')}"
else:
    pdb_prot = pdb_hdr_d.get("name")

pdb_prot

'"Structure Of Cyanobacterial Photosystem I-Isia Supercomplex"\nNOTE: 6kig-assembly1.cif converted to pdb by mcce_protinfo; truncated: true'

In [126]:
!head {pdb_from_cif_fp}

HEADER    6kig-assembly1.cif converted to pdb by MCCE_ProtInfo; truncated: True
TITLE     "Structure of cyanobacterial photosystem I-IsiA supercomplex" 
ATOM      1  N   LYS A  13     197.542 182.828 269.142  1.00 29.33           N  
ATOM      2  CA  LYS A  13     198.177 183.951 268.467  1.00 29.33           C  
ATOM      3  C   LYS A  13     197.139 184.965 268.013  1.00 29.33           C  
ATOM      4  O   LYS A  13     197.362 186.172 268.086  1.00 29.33           O  
ATOM      5  CB  LYS A  13     198.995 183.463 267.275  1.00 29.33           C  
ATOM      6  CG  LYS A  13     200.252 184.272 267.040  1.00 29.33           C  
ATOM      7  CD  LYS A  13     201.182 184.177 268.235  1.00 29.33           C  
ATOM      8  CE  LYS A  13     202.329 185.155 268.113  1.00 29.33           C  


In [19]:
log_specs = log_parser.blocks_specs
ptp(log_specs)

{1: LogHdr(idx=1,
           hdr='   Rename residue and atom names...',
           rpt_hdr='Renamed:',
           line_start='   Renaming ',
           skip_lines=None,
           debuglog=False),
 2: LogHdr(idx=2,
           hdr='   Identify NTR and CTR...',
           rpt_hdr='Termini:',
           line_start='      Labeling ',
           skip_lines=None,
           debuglog=False),
 3: LogHdr(idx=3,
           hdr='   Label backbone, sidechain and altLoc conformers...',
           rpt_hdr='Labeling:',
           line_start='      Labeling ',
           skip_lines=('   Creating temporary parameter file for unrecognized '
                       'residue...',
                       '   Trying labeling again...',
                       '   Try delete this entry and run MCCE again',
                       '   Error! premcce_confname()',
                       ' is already loaded somewhere else.'),
           debuglog=False),
 4: LogHdr(idx=4,
           hdr='   Load pdb lines into data s

In [96]:
# single pdb report:

pdb = Path("1fat.pdb")
args = {"input_pdb":pdb, "fetch":False}

info.get_single_pdb_report(args)

rptmd = Path(pdb).resolve().parent.joinpath("ProtInfo.md")

In [97]:
Markdown(filename=rptmd)

---
# 1fat
## ParsedStructure
### Chains
  * <strong>4 </strong> : A, B, C, D

### Residues
  - 956

### Waters
  - 4

### Warnings
  - <strong><font color='red'>Discontinuity</font> </strong>
    - Chain A: ['Line 7975', 'Line 8039']
    - Chain B: ['Line 7991', 'Line 8043']
    - Chain C: ['Line 8007', 'Line 8047']
    - Chain D: ['Line 8023', 'Line 8051']

## MCCE.Step1
### Renamed:
  - "CA    CA A 255" to "CA   _CA A 255"
  - "CA    CA B 255" to "CA   _CA B 255"
  - "CA    CA C 255" to "CA   _CA C 255"
  - "CA    CA D 255" to "CA   _CA D 255"
  - "MN    MN A 254" to "MN   _MN A 254"
  - "MN    MN B 254" to "MN   _MN B 254"
  - "MN    MN C 254" to "MN   _MN C 254"
  - "MN    MN D 254" to "MN   _MN D 254"

### Termini:
  * <strong>NTR </strong> : "SER A   1", "ASN A  38", "SER B   1", "SER C   1", "ASN C  38", "SER D   1", "ASN D  38"
  * <strong>CTR </strong> : "ASN A  36", "SER A 233", "SER B 233", "ASN C  36", "SER C 233", "LEU D  35", "SER D 233"

### Labeling:
  - <strong><font color='red'>Generic topology file created for:</font> </strong>
  - NAG::  https://pubchem.ncbi.nlm.nih.gov/#query=NAG&tab=substance
  - _MN::  https://pubchem.ncbi.nlm.nih.gov/#query=MN&tab=substance
  - _CA::  https://pubchem.ncbi.nlm.nih.gov/#query=CA&tab=substance

### Free Cofactors:
  - Species and properties with assigned default values in debug.log:
  - 	NAGBK: ['VDW_RAD', 'VDW_EPS']
  - 	_MNBK: ['VDW_RAD', 'VDW_EPS']
  - 	_CABK: ['VDW_RAD', 'VDW_EPS']

### Missing Heavy Atoms:
  - OXT of conf CTR01 in "CTR A  36".
  - OXT of conf CTR01 in "CTR A 233".
  - OXT of conf CTR01 in "CTR B 233".
  - OXT of conf CTR01 in "CTR C  36".
  - OXT of conf CTR01 in "CTR C 233".
  - OXT of conf CTR01 in "CTR D  35".
  - OXT of conf CTR01 in "CTR D 233".

### Distance Clashes:
<details><summary>Clashes found</summary>

  -    d= 1.53: " CA  NTR A   1" to " CB  SER A   1"
  -    d= 1.45: " ND2 ASN A  12" to " C1  NAG A 253"
  -    d= 1.53: " CA  NTR A  38" to " CB  ASN A  38"
  -    d= 1.52: " CA  NTR B   1" to " CB  SER B   1"
  -    d= 1.48: " ND2 ASN B  12" to " C1  NAG B 253"
  -    d= 1.53: " CA  NTR C   1" to " CB  SER C   1"
  -    d= 1.45: " ND2 ASN C  12" to " C1  NAG C 253"
  -    d= 1.52: " CA  NTR C  38" to " CB  ASN C  38"
  -    d= 1.87: " OD1 ASN C 128" to "CA   _CA C 255"
  -    d= 1.82: " NE2 HIS C 137" to "MN   _MN C 254"
  -    d= 1.54: " CA  NTR D   1" to " CB  SER D   1"
  -    d= 1.43: " ND2 ASN D  12" to " C1  NAG D 253"
  -    d= 1.55: " CA  NTR D  38" to " CB  ASN D  38"
  -    d= 1.70: "MN   _MN A 254" to " O   HOH A 307"
  -    d= 1.46: "MN   _MN A 254" to " O   HOH A 308"
  -    d= 1.99: "CA   _CA A 255" to " O   HOH A 306"
  -    d= 1.45: "MN   _MN B 254" to " O   HOH B 304"
  -    d= 1.69: "CA   _CA B 255" to " O   HOH B 301"
  -    d= 1.55: "MN   _MN C 254" to " O   HOH C 316"
  -    d= 1.50: "MN   _MN D 254" to " O   HOH D 311"
  -    d= 1.52: "MN   _MN D 254" to " O   HOH D 312"
</details>

---


In [83]:
prot_d, s1_d = info.collect_info(pdb)
prot_d

s1_d

{'1fat': {'ParsedStructure': {'Chains': [(4, ['A', 'B', 'C', 'D'])],
   'Residues': [956],
   'Waters': [4],
   'Warnings': {'Discontinuity': {'Chain A': ['Line 7975', 'Line 8039'],
     'Chain B': ['Line 7991', 'Line 8043'],
     'Chain C': ['Line 8007', 'Line 8047'],
     'Chain D': ['Line 8023', 'Line 8051']}}}}}

{'1fat': {'MCCE.Step1': {'Renamed:': ['"CA    CA A 255" to "CA   _CA A 255"',
    '"CA    CA B 255" to "CA   _CA B 255"',
    '"CA    CA C 255" to "CA   _CA C 255"',
    '"CA    CA D 255" to "CA   _CA D 255"',
    '"MN    MN A 254" to "MN   _MN A 254"',
    '"MN    MN B 254" to "MN   _MN B 254"',
    '"MN    MN C 254" to "MN   _MN C 254"',
    '"MN    MN D 254" to "MN   _MN D 254"'],
   'Termini:': [('NTR',
     ['"SER A   1"',
      '"ASN A  38"',
      '"SER B   1"',
      '"SER C   1"',
      '"ASN C  38"',
      '"SER D   1"',
      '"ASN D  38"']),
    ('CTR',
     ['"ASN A  36"',
      '"SER A 233"',
      '"SER B 233"',
      '"ASN C  36"',
      '"SER C 233"',
      '"LEU D  35"',
      '"SER D 233"'])],
   'Labeling:': ['Generic topology file created for:',
    'NAG::  https://pubchem.ncbi.nlm.nih.gov/#query=NAG&tab=substance',
    '_MN::  https://pubchem.ncbi.nlm.nih.gov/#query=MN&tab=substance',
    '_CA::  https://pubchem.ncbi.nlm.nih.gov/#query=CA&tab=substance'],
   'Load

---
---

---
# Test cli

In [4]:
from argparse import ArgumentParser, ArgumentError, RawDescriptionHelpFormatter, Namespace


In [5]:
cli_parser = cli.bench_parser()